In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.patches import Rectangle
from torch.utils.data import Dataset, DataLoader

In [2]:
class NGSIMDataset(Dataset):
    def __init__(self, root_dir, dataset_path, past_frames=20, future_frames=20):
        self.root_dir = root_dir
        self.dataset_path = dataset_path
        self.past_frames = past_frames
        self.future_frames = future_frames
        self.dataset_df = pd.read_csv(self.dataset_path, nrows=10000)
        self.num_rows = len(self.dataset_df)
        self.dataset_names = self.dataset_df["Dataset_Name"].unique().tolist()
        self.column_names = ["Vehicle_ID", "Frame_ID", "Total_Frames", "Local_X", "Local_Y", "Lane_ID"]
        self.data_frame_dict = {}
        self.unique_vehicles_dict = {}
        self.unique_frames_dict = {}
        self.vehicles_data_dict = {}
        self.frames_data_dict = {}
        self.initialize()
    
    def initialize(self):
        for dataset_name in self.dataset_names:
            traj_df = pd.read_csv(os.path.join(self.root_dir, dataset_name), usecols=self.column_names)
            frame_ids_list = self.get_unique_frame_ids(traj_df)
            vehicle_ids_list = self.get_unique_vehicle_ids(traj_df)
            frame_data_dict = self.get_frame_data(traj_df, frame_ids_list)            
            vehicle_data_dict = self.get_vehicle_data(traj_df, vehicle_ids_list)
            
            self.data_frame_dict[dataset_name] = traj_df
            self.frames_data_dict[dataset_name] = frame_data_dict
            self.unique_frames_dict[dataset_name] = frame_ids_list
            self.vehicles_data_dict[dataset_name] = vehicle_data_dict
            self.unique_vehicles_dict[dataset_name] = vehicle_ids_list
    
    def __len__(self):
        return len(self.dataset_df)
    
    def get_unique_vehicle_ids(self, traj_df):
        return traj_df["Vehicle_ID"].unique().tolist()
    
    def get_unique_frame_ids(self, traj_df):
        return traj_df["Frame_ID"].unique().tolist()

    def get_vehicle_data(self, traj_df, vehicle_ids_list):
        vehicle_data_dict = {}
        for vehicle_id in vehicle_ids_list:
            vehicle_data_dict[vehicle_id] = traj_df[traj_df["Vehicle_ID"] == vehicle_id].reset_index(drop=True)
        return vehicle_data_dict

    def get_frame_data(self, traj_df, frame_ids_list):
        frame_data_dict = {}
        for frame_id in frame_ids_list:
            frame_data_dict[frame_id] = traj_df[traj_df["Frame_ID"] == frame_id].reset_index(drop=True)
        return frame_data_dict
    
    def get_frame_bounds(self, traj_df, frame_id):
        # Get index of the current row in the vehicle's trajectory dataframe
        traj_index = traj_df[traj_df["Frame_ID"] == frame_id].index[0]
        lower_bound = max(0, traj_index - self.past_frames) # (t - k)s in the past
        upper_bound = min(len(traj_df) - 1, traj_index + self.future_frames) # (t + k')s into the future
        return traj_index, lower_bound, upper_bound

    def generate_trajectories(self, traj_df, frame_id):
        traj_index, lower_bound, upper_bound = self.get_frame_bounds(traj_df, frame_id)
        vehicle_traj = traj_df[["Local_X", "Local_Y"]].to_numpy()
        past_traj = vehicle_traj[lower_bound:traj_index + 1, :]
        future_traj = vehicle_traj[traj_index + 1:upper_bound + 1, :]
        return past_traj, future_traj
    
    def get_velocities(self, traj_df, frame_id):
        traj_index, lower_bound, upper_bound = self.get_frame_bounds(traj_df, frame_id)
        pos_past = traj_df.iloc[lower_bound]["Local_Y"]
        pos_cur = traj_df.iloc[traj_index]["Local_Y"]
        pos_future = traj_df.iloc[upper_bound]["Local_Y"]
        vel_hist = (pos_cur - pos_past) / (traj_index - lower_bound)
        vel_future = (pos_future - pos_cur) / (upper_bound - traj_index)
        return vel_hist, vel_future
        
    def get_lateral_manoeuvres(self, traj_df, frame_id):
        lateral_manoeuvres_id = None
        traj_index, lower_bound, upper_bound = self.get_frame_bounds(traj_df, frame_id)
        upper_lane_id = traj_df.iloc[upper_bound]["Lane_ID"]
        lower_lane_id = traj_df.iloc[lower_bound]["Lane_ID"]
        cur_lane_id = traj_df.iloc[traj_index]["Lane_ID"]

        if upper_lane_id > cur_lane_id or cur_lane_id > lower_lane_id:
            lateral_manoeuvres_id = 3
        elif upper_lane_id < cur_lane_id or cur_lane_id < lower_lane_id:
            lateral_manoeuvres_id = 2
        else:
            lateral_manoeuvres_id = 1
        return lateral_manoeuvres_id

    def get_longitudinal_manoeuvres(self, traj_df, frame_id):
        longitudinal_manoeuvres_id = None
        traj_index, lower_bound, upper_bound = self.get_frame_bounds(traj_df, frame_id)
        if upper_bound == traj_index or lower_bound == traj_index:
            longitudinal_manoeuvres_id = 1
        else:
            vel_hist, vel_future = self.get_velocities(traj_df, frame_id)
            if vel_future / vel_hist < 0.8:
                longitudinal_manoeuvres_id = 2
            else:
                longitudinal_manoeuvres_id = 3
        return longitudinal_manoeuvres_id
    
    def get_manoeuvres(self, traj_df, frame_id):
        lateral_manoeuvres_id = self.get_lateral_manoeuvres(traj_df, frame_id)
        longitudinal_manoeuvres_id = self.get_longitudinal_manoeuvres(traj_df, frame_id)
        return lateral_manoeuvres_id, longitudinal_manoeuvres_id    

    def get_pos(self, traj_df, frame_id):
        cur_frame = traj_df[traj_df["Frame_ID"] == frame_id]
        vehicle_pos = np.array([[cur_frame.iloc[0]["Local_X"], cur_frame.iloc[0]["Local_Y"]]])
        return vehicle_pos

    def get_longitudinal_dis(self, vehicle_pos, ego_pos):
        return abs(vehicle_pos[0, 1] - ego_pos[0, 1])
    
    def get_scene_data(self, dataset_name, row):
        ego_id = row["Vehicle_ID"]
        frame_id = row["Frame_ID"]
        ego_pos = np.array([row["Local_X"], row["Local_Y"]]).reshape(1, 2)
        
        ego_vehicle_traj_df = self.vehicles_data_dict[dataset_name][ego_id]
        frame_data_df = self.frames_data_dict[dataset_name][frame_id]

        ego_vel_hist, ego_vel_fut = self.get_velocities(ego_vehicle_traj_df, frame_id)
        ego_vehicle_past_traj, ego_vehicle_future_traj = self.generate_trajectories(ego_vehicle_traj_df, frame_id)

        # Get Trajectory of other vehicles
        other_vehicle_ids_list = frame_data_df["Vehicle_ID"].unique().tolist()
        other_vehicle_past_traj_list, other_vehicle_future_traj_list = [], []
        other_vehicle_vel_hist_list, other_vehicle_vel_fut_list = [], []        
        other_vehicle_pos_list = []
        for other_vehicle_id in other_vehicle_ids_list:
            if other_vehicle_id != ego_id:
                other_vehicle_traj_df = self.vehicles_data_dict[dataset_name][other_vehicle_id]
                other_vehicle_pos = self.get_pos(other_vehicle_traj_df, frame_id)
                longitudinal_dis = self.get_longitudinal_dis(other_vehicle_pos, ego_pos)

                # Only consider vehicles that are withing some distance of ego-vehicle
                if longitudinal_dis <= 90:
                    other_vehicle_vel_hist, other_vehicle_vel_fut = self.get_velocities(other_vehicle_traj_df, frame_id)
                    other_vehicle_past_traj, other_vehicle_future_traj = self.generate_trajectories(other_vehicle_traj_df, frame_id)
                    other_vehicle_pos_list.append(other_vehicle_pos)                    
                    other_vehicle_vel_fut_list.append(other_vehicle_vel_fut)                    
                    other_vehicle_vel_hist_list.append(other_vehicle_vel_hist)
                    other_vehicle_past_traj_list.append(other_vehicle_past_traj)
                    other_vehicle_future_traj_list.append(other_vehicle_future_traj)
        
        num_vehicles = len(other_vehicle_pos_list)
        lateral_manoeuvres_id, longitudinal_manoeuvres_id = self.get_manoeuvres(ego_vehicle_traj_df, frame_id)
        return num_vehicles, ego_pos, other_vehicle_pos_list, ego_vehicle_past_traj, ego_vehicle_future_traj, other_vehicle_past_traj_list, other_vehicle_future_traj_list, lateral_manoeuvres_id, longitudinal_manoeuvres_id, ego_vel_hist, ego_vel_fut, other_vehicle_vel_hist_list, other_vehicle_vel_fut_list
    
    def __getitem__(self, idx):
        row = self.dataset_df.iloc[idx]
        dataset_name = row["Dataset_Name"]
        frame_id = row["Frame_ID"]
        vehicle_id = row["Vehicle_ID"]
        
        traj_df = self.data_frame_dict[dataset_name]
        traj_df_row = traj_df[(traj_df["Frame_ID"] == frame_id) & (traj_df["Vehicle_ID"] == vehicle_id)].iloc[0]
        return self.get_scene_data(dataset_name, traj_df_row)

In [3]:
ngsim_dataset = NGSIMDataset(root_dir="../../../datasets/raw_data/", dataset_path="../../../datasets/ngsim_dataset/ngsim_train.csv", past_frames=32, future_frames=48)

In [4]:
def visualize(ego_pos, other_vehicle_pos_list, ego_vehicle_past_traj, ego_vehicle_future_traj, other_vehicle_past_traj_list, other_vehicle_future_traj_list, traj_num):
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    ############################################# Plot Scene ##################################################
    # Plot road boundaries
    axs[0].axhline(y=-3.5)
    axs[0].axhline(y=75.0)
    
    # Plot lane boundaries
    axs[0].axhline(y=11.5, color='k', linestyle='-.', linewidth=2)
    axs[0].axhline(y=23.5, color='k', linestyle='-.', linewidth=2)
    axs[0].axhline(y=35.0, color='k', linestyle='-.', linewidth=2)
    axs[0].axhline(y=48.0, color='k', linestyle='-.', linewidth=2)
    axs[0].axhline(y=60.0, color='k', linestyle='-.', linewidth=2)
    
    _ = axs[0].set_xlabel('Longitude (Feet)', fontsize=12)
    _ = axs[0].set_ylabel('Lateral (Feet)', fontsize=12)
        
    # While plotting we need to flip X and Y's
    _ = axs[0].scatter(ego_pos[:, 1], ego_pos[:, 0])
    _ = axs[0].scatter(other_vehicle_pos_list[:, 1], other_vehicle_pos_list[:, 0])
    
    # Add rectangles for ego vehicle
    width, height = 15, 5
    _ = axs[0].add_patch(Rectangle(xy=(ego_pos[0, 1] - width/2, ego_pos[0, 0] - height/2), width=width, height=height, linewidth=3, color='green', fill=False))
    # Add rectangles for other vehicles
    for (a_x, a_y) in other_vehicle_pos_list:
        _ = axs[0].add_patch(Rectangle(xy=(a_y-width/2, a_x-height/2) , width=width, height=height, linewidth=1, color='blue', fill=False))
    
    _ = axs[0].set_xlim([ego_pos[0, 1] - 240, ego_pos[0, 1] + 240])
    _ = axs[0].set_ylim([-7.5, 80])
    _ = axs[0].set_title("Current Frame")
    
    ######################################### Plot Trajectories ###############################################
    # Plot road boundaries
    axs[1].axhline(y=-3.5)
    axs[1].axhline(y=75.0)
    
    # Plot lane boundaries
    axs[1].axhline(y=11.5, color='k', linestyle='-.', linewidth=2)
    axs[1].axhline(y=23.5, color='k', linestyle='-.', linewidth=2)
    axs[1].axhline(y=35.0, color='k', linestyle='-.', linewidth=2)
    axs[1].axhline(y=48.0, color='k', linestyle='-.', linewidth=2)
    axs[1].axhline(y=60.0, color='k', linestyle='-.', linewidth=2)
    
    _ = axs[1].set_xlabel('Longitude (Feet)', fontsize=12)
    _ = axs[1].set_ylabel('Lateral (Feet)', fontsize=12)        
    
    # While plotting we need to flip X and Y's    

    # Plot past trajectories
    _ = axs[1].scatter(ego_vehicle_past_traj[:, 1], ego_vehicle_past_traj[:, 0], color='gray', linewidth=0.2, alpha=0.5)
    _ = axs[1].scatter(other_vehicle_past_traj_list[:, 1], other_vehicle_past_traj_list[:, 0], color='gray', linewidth=0.2, alpha=0.5)

    # Plot future trajectories
    _ = axs[1].scatter(ego_vehicle_future_traj[:, 1], ego_vehicle_future_traj[:, 0], color='green', linewidth=0.2, alpha=0.5)
    _ = axs[1].scatter(other_vehicle_future_traj_list[:, 1], other_vehicle_future_traj_list[:, 0], color='red', linewidth=0.2, alpha=0.5)
    
    # Plot ego vehicle and other vehicles position
    _ = axs[1].scatter(ego_pos[:, 1], ego_pos[:, 0], color='black')
    _ = axs[1].scatter(other_vehicle_pos_list[:, 1], other_vehicle_pos_list[:, 0], color='black')
    
    # Add rectangles for ego vehicle
    width, height = 15, 5
    _ = axs[1].add_patch(Rectangle(xy=(ego_pos[0, 1] - width/2, ego_pos[0, 0] - height/2), width=width, height=height, linewidth=3, color='green', fill=False))
    # Add rectangles for other vehicles
    for (a_x, a_y) in other_vehicle_pos_list:
        _ = axs[1].add_patch(Rectangle(xy=(a_y-width/2, a_x-height/2) , width=width, height=height, linewidth=2, color='blue', fill=False))
    
    
    _ = axs[1].set_xlim([ego_pos[0, 1] - 240, ego_pos[0, 1] + 240])
    _ = axs[1].set_ylim([-7.5, 80])
    _ = axs[1].set_title("Past and Future Trajectories")
    
    plt.savefig('./scene_plots/{}.png'.format(traj_num))
    plt.close()

In [7]:
scenes_data = []
for i in range(50, len(ngsim_dataset), 100):
    num_vehicles, ego_pos, other_vehicle_pos_list, ego_vehicle_past_traj, ego_vehicle_future_traj, other_vehicle_past_traj_list, other_vehicle_future_traj_list, _, _, _, _, _, _ = ngsim_dataset[i]
    ego_traj = np.concatenate([ego_vehicle_past_traj, ego_vehicle_future_traj])[1::4]
    cur_scene_data = []
    cur_scene_data.append(ego_traj)
    for j in range(num_vehicles):
        traj = np.concatenate([other_vehicle_past_traj_list[j], other_vehicle_future_traj_list[j]])[1::4]
        cur_scene_data.append(traj)
    
    flag = 1
    for traj in cur_scene_data:
        if traj.shape[0] != 20:
            flag = 0
            break
    if flag == 0:
        continue
    scenes_data.append(cur_scene_data)
    other_vehicle_pos_list = np.concatenate(other_vehicle_pos_list)
    other_vehicle_past_traj_list = np.concatenate(other_vehicle_past_traj_list)
    other_vehicle_future_traj_list = np.concatenate(other_vehicle_future_traj_list)
    visualize(ego_pos, other_vehicle_pos_list, ego_vehicle_past_traj, ego_vehicle_future_traj, other_vehicle_past_traj_list, other_vehicle_future_traj_list, i)
    if i >= 100000:
        break

In [8]:
np.save('scene_data.npy', scenes_data)